In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import numpy as np
from scipy.io import loadmat,savemat

In [ ]:
#for docomo, 9 CUDAs are available in total
cuda_num = 0
num_device = torch.cuda.device_count()
print('There are '+str(num_device)+' GPU(s). We will use CUDA '+str(cuda_num)+' in this program.')
device = torch.device('cuda:'+str(cuda_num))

In [ ]:
#assign parameter values
test_rate = .2
lr = 1e-3
min_epoch,max_epoch = 30,3000
num_BS,num_beam = 4,512

In [ ]:
#load datasets from files
in_set_file = loadmat('DLCB_Dataset/DLCB_input.mat')
in_set = in_set_file['DL_input']#in_set.shape=(54481,256)
out_set_file = loadmat('DLCB_Dataset/DLCB_output.mat')
out_set = out_set_file['DL_output']#out_set.shape=(54481,2048)

In [ ]:
#define a Network
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        input_nodes,output_nodes,middle_nodes=512,512,512
        self.fc1 = nn.Linear(input_nodes,middle_nodes)
        self.fc2 = nn.Linear(middle_nodes,middle_nodes)
        self.fc3 = nn.Linear(middle_nodes,middle_nodes)
        self.fc4 = nn.Linear(middle_nodes,middle_nodes)
        self.fc5 = nn.Linear(middle_nodes,output_nodes)
        
    def forward(self,x):
        # 4 hidden layers with each fc + relu + dropout
        x = x.to(device)
        dropout_rate=0.05
        x = F.dropout(F.relu(self.fc1(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc2(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc3(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc4(x)),p = dropout_rate,training = True)
        # 1 output layer with fc + relu
        x = F.relu(self.fc5(x))
        return x  
net = Net().to(device)

#define Loss Function and Optimization method
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(net.parameters(),lr=lr)

In [ ]:
class DataSetGenerator(data.Dataset):
    def __init__(self,input_set,output_set):
        self.input_set = input_set
        self.output_set = output_set
    
    def __getitem__(self,index):
        x = torch.from_numpy(self.input_set[index])
        label = torch.from_numpy(self.output_set[index])
        x,label=x.type(torch.float32),label.type(torch.float32)
        return x,label
    
    def __len__(self):
        return self.input_set.shape[0]

In [ ]:
#get num_total_user from in_set
def getTotalUser(in_set):
    return in_set.shape[0]

In [ ]:
#get DL_size_list, 17 percentages are fixed according to the author
def getDLSizeList():
    DL_pr_list = np.array([.001,.05,.1,.15,.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8])
    return np.floor(DL_pr_list*num_total_user).astype(np.int64).tolist()

In [ ]:
#get num_train and num_test, both are lists according to DL_size_list, num_total_user, test_rate
def getAmount(DL_size_list,num_total_user,test_rate):
    num_train=np.floor(np.asarray(DL_size_list)*.8).astype(np.int64).tolist()
    num_test=int(num_total_user*test_rate)
    return num_train,num_test

In [ ]:
#specifically for the i-th DL_size, return its in_train_pre,in_test_pre,out_train_pre,out_test_pre(all before any adjustments)
def eachInOutPre(i,num_train,num_test,num_total_user,in_set,out_set):
    train_index = np.random.choice(range(0,num_total_user),size=num_train[i],replace=False)
    rem_index = set(range(0,num_total_user))-set(train_index)
    test_index = list(set(np.random.choice(list(rem_index),size=num_test,replace=False)))
    in_train_pre = in_set[train_index]
    in_test_pre = in_set[test_index]
    out_train_pre = out_set[train_index]
    out_test_pre = out_set[test_index]
    return in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index

In [ ]:
#arrange input into real part and imaginary part one by one
def arrangeInput(lst):
    re=np.array(lst).real
    im=np.array(lst).imag
    return np.stack((re,im),axis=2).reshape(lst.shape[0],lst.shape[1]*2)

In [ ]:
#specificially for i BS(s) in usage, divide output columns by num_beam
def divideOutput(i,num_beam,lst):
    return np.array(lst)[:,i*num_beam:(i+1)*num_beam]

In [ ]:
#for the i_DL-th DL_size and i_Out BS(s), generate and load datasets, where i_DL 0-16, i_Out 0-3
def getLoader(i_DL,i_Out):
    in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index = eachInOutPre(i_DL,num_train,num_test,num_total_user,in_set,out_set)
    in_train = arrangeInput(in_train_pre)
    in_test = arrangeInput(in_test_pre)
    out_train = divideOutput(i_Out,num_beam,out_train_pre)
    out_test = divideOutput(i_Out,num_beam,out_test_pre)

    train_generator = DataSetGenerator(input_set = in_train, output_set = out_train)
    train_loader = data.DataLoader(dataset=train_generator, batch_size=128,shuffle=True)
    
    test_generator = DataSetGenerator(input_set = in_test, output_set = out_test)
    test_loader = data.DataLoader(dataset=test_generator, batch_size=64,shuffle=True)
    return train_loader,test_loader,in_test,out_test

In [ ]:
num_total_user = getTotalUser(in_set)
DL_size_list = getDLSizeList()
num_train,num_test = getAmount(DL_size_list,num_total_user,test_rate)
best_net = {}

In [ ]:
for i_DL in range(len(DL_size_list)):
    for i_Out in range(0,num_BS,1):
        net.to(device)
        net.train()
        train_loader,test_loader,in_test,out_test = getLoader(i_DL,i_Out)
        test_loss=[]#average test loss of an epoch
        min_loss=1e5#a large number
        
        for epoch in range(max_epoch):
            for batch_idx,(x,label) in enumerate(train_loader):
                x = x.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                y_hat = net(x)
                loss = criterion(label.to(torch.float32),y_hat.to(torch.float32))
                loss.backward()
                optimizer.step()

            for batch_idx,(x,label) in enumerate(test_loader):
                with torch.no_grad():
                    x = x.to(device)
                    label = label.to(device)
                    optimizer.zero_grad()
                    y_hat = net(x)
                    test_loss.append(1e4*criterion(label.to(torch.float32),y_hat.to(torch.float32)))
            batch_loss = sum(test_loss)/len(test_loss)
            if (epoch > min_epoch)and(batch_loss < min_loss):
                min_loss = batch_loss
                print('Min_loss is '+str(min_loss.items())+'*1e-4 at Epoch '+str(epoch))
                saveNet='DLCB_code_output/cache/DL'+str(i_DL)+'Out'+str(i_Out)+'.pth'
                torch.save({'epoch':epoch,
                           'model_state_dict':net.state_dict(),
                           'optimizer_state_dict':optimizer.state_dict(),
                          'loss':loss},saveNet)               
        if epoch==(max_epoch-1):
            print('End Train and Test of DL size {} and BS {} successfully.'.format(DL_size_list[i_DL],i_Out+1))

In [ ]:
#load previous net
def loadSave(i_DL,i_Out):
    saveNet='DLCB_code_output/cache/DL'+str(i_DL)+'Out'+str(i_Out)+'.pth'
    return torch.load(saveNet,map_location=device)

In [ ]:
#continue to train and test under certain circumstances, train another 1000 epochs
def trainContinue(i_DL,i_Out):
    checkpoint=loadSave(i_DL,i_Out)
    org_epoch=checkpoint['epoch']
    extra_epoch=1e3
    min_loss=checkpoint['loss']
    net.load_state_dict=checkpoint['model_state_dict']
    optimizer.load_state_dict=checkpoint['optimizer']
    net.to(device)
    net.train()
    train_loader,test_loader,in_test,out_test = getLoader(i_DL,i_Out)
    test_loss=[]#average test loss of an epoch
    for epoch in range(extra_epoch):
            
        for batch_idx,(x,label) in enumerate(train_loader):
            x = x.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            y_hat = net(x)
            loss = criterion(label.to(torch.float32),y_hat.to(torch.float32))
            loss.backward()
            optimizer.step()

        for batch_idx,(x,label) in enumerate(test_loader):
            with torch.no_grad():
                x = x.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                y_hat = net(x)
                test_loss.append(1e4*criterion(label.to(torch.float32),y_hat.to(torch.float32)))
        batch_loss = sum(test_loss)/len(test_loss)
        if batch_loss < min_loss:
            min_loss = batch_loss
            print('Min_loss is '+str(min_loss)+'*1e-4 at Epoch '+str(epoch+org_epoch))
            torch.save({'epoch':epoch+org_epoch,
                        'model_state_dict':net.state_dict(),
                        'optimizer_state_dict':optimizer.state_dict(),
                        'loss':loss},saveNet)               
    if epoch==(max_epoch-1):
        print('Another 1000 epochs of DL size {} and BS {} successfully.'.format(DL_size_list[i_DL],i_Out+1))

In [ ]:
#the last Phase
DL_Result={}
for i_DL in range(len(DL_size_list)):
    y_pred = np.zeros((num_test,num_beam))
    for i_Out in range(0,num_BS,1):
        _,test_loader,_,out_test = getLoader(i_DL,i_Out)
        checkpoint=loadSave(i_DL,i_Out)
        net.load_state_dict(checkpoint['model_state_dict'])
        net.to(device)
        net.eval()
        for batch_idx,(x,label) in enumerate(test_loader):
            with torch.no_grad():
                y_hat = net(x)
                y_hat_np = y_hat.cpu().numpy()
                for i in range(y_hat_np.shape[0]):
                    y_pred[batch_idx*10+i] = y_hat_np[i]
        DL_Result['TX'+str(i_Out+1)+'Pred_Beams'] = y_pred[:,:]
        DL_Result['TX'+str(i_Out+1)+'Opt_Beams'] = out_test[:,:]
    _,_,_,_,test_index = eachInOutPre(i_DL,num_train,num_test,num_total_user,in_set,out_set)
    DL_Result['user_index'] = test_index
    savemat('DLCB_code_output/DL_Result'+str(i_DL+1)+'.mat',DL_Result)
    print('Savemat succeeds.\n')